<a href="https://colab.research.google.com/github/OsvaldoUfla/Dijkstra-Codigo-exemplo/blob/main/07_02_Dijkstra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo de Dijkstra

## Motivação

O esquema a seguir representa os caminhos que ligam diversas localidades por onde devem passar o mosqueteiro D'Artagnan, que está repleto de emboscadas. Os números representam a probabilidade $\frac{x}{10}$ de sucesso de ultrapassar os trechos. Por exemplo, entre os vértices $2$ e $4$, a probabilidade de sucesso é de $70\%$. Qual é a probabilidade de sucesso de D'Artagnan de $1$ até os outros vértices? 

## Implementação

O problema acima pode ser resolvido através do Algoritmo de Dijkstra, pois se trata de uma adaptação do problema de caminho mais curto de *fonte única*. 

$G=(V,E)$ trata-se de um grafo ponderado e orientado, em que os pesos dos arcos indicam a probabilidade de sucesso de ultrapassar cada trecho. Como Dijkstra visa minimizar o caminho mais curto, propõe-se a resolução do problema minizando o fracasso, dado por $1-\frac{x}{10}$. O vértice de origem é dado pelo vértice $1 \in V$.

Abaixo, segue uma sugestão de código para resolver o problema. **Note:** como pretende-se minimizar a probabilidade de fracasso, a atualização do valor da estimativa do menor caminho da origem  passando pelo arco $(u,v)$ deve considerar o produto de $d[u]$ por $w_{uv}$, em que $w_{uv}$ é o peso do arco $(u,v)$.

In [ ]:
%%writefile dijkstra.cpp
#include<iostream>
#include<vector>
#include<queue>
#include<utility>
#include<functional>
using namespace std;

/*
 * Variaveis globais
 */

// lista de adjacencia
vector<pair<int, double>>* LA;

// numero de vertices do grafo
int n;

// numeor de arestas do grafo
int m;

// distancia da origem "org" a cada vertice do grafo
vector<double> d;

int dijkstra(int org)
{
    d.assign(n, 1.0);
    
    // a distance da origem "org" eh sempre zero
    d[org] = 1.0;
    
    // heap que auxilia na obtencao do vertice com maior prioridade, a cada iteracao
    priority_queue< pair<double, int>, vector<pair<double, int>>, greater<pair<double, int>> > heap;

    // primeiro par inserido na heap: "org" com custo zero
    heap.push(make_pair(1.0, org));
 
    vector<bool> visitado;
    visitado.assign(n, false);
 
    // o algoritmo para quando a heap estiver vazia
    while(!heap.empty())
    {
        pair<double, int> vertice = heap.top();
        heap.pop();

        double distancia = vertice.first;
        int u = vertice.second;
     
        if(visitado[u]) // "u" jah foi explorado
          continue;
     
        visitado[u] = true;
     
        double custo;
        for(int j = 0; j < (int) LA[u].size(); j++)
        {
            pair<int, double> vizinho = LA[u][j];
            int v = vizinho.first;
            double prob = vizinho.second;
         
            // tentativa de melhorar a estimativa de menor caminho da origem ao vertice v
            custo = d[u] * prob;
            if(custo < d[v]) 
            { 
                d[v] = custo; 
                heap.push(make_pair(d[v], v)); 
            }
        }
    }
}

int main()
{
    cin >> n >> m;
   
    LA = new vector<pair<int, double>>[n];
    int u, v;
    double p;
    for(int i = 0; i < m; i++)
    {
        cin >> u >> v; 
        cin >> p;
        u--;
        v--;
        LA[u].push_back(make_pair(v, 1.0 - p/10.0));
    }
 
    for(int i = 0; i < n; i++)
    {
        cout << "vertice " << i << ": ";
        for(int j = 0; j < (int) LA[i].size(); j++)
        {
            cout << "(" << LA[i][j].first << ", " << LA[i][j].second << ") ";
        }
        cout << endl;
    }

    dijkstra(0);
 
    for(int i = 0; i < n; i++)
      cout << "d[" << i + 1 << "]: " << 1.0 - d[i] << endl;

    return 0;
}

Overwriting dijkstra.cpp


*Compilando o código*

In [ ]:
%%script bash
g++ dijkstra.cpp -o dijkstra

*Executando o código*

In [ ]:
%%script bash
echo -e "12 17\n1 2 9\n1 3 8\n2 4 7\n3 4 7\n3 5 8\n4 6 3\n4 7 9\n5 6 5\n5 7 6\n6 8 1\n6 9 4\n7 9 1\n8 10 2\n9 10 5\n9 11 9\n10 12 6\n11 12 1\n" > 01.in
./dijkstra < 01.in

vertice 0: (1, 0.1) (2, 0.2) 
vertice 1: (3, 0.3) 
vertice 2: (3, 0.3) (4, 0.2) 
vertice 3: (5, 0.7) (6, 0.1) 
vertice 4: (5, 0.5) (6, 0.4) 
vertice 5: (7, 0.9) (8, 0.6) 
vertice 6: (8, 0.9) 
vertice 7: (9, 0.8) 
vertice 8: (9, 0.5) (10, 0.1) 
vertice 9: (11, 0.4) 
vertice 10: (11, 0.9) 
vertice 11: 
d[1]: 0
d[2]: 0.9
d[3]: 0.8
d[4]: 0.97
d[5]: 0.96
d[6]: 0.98
d[7]: 0.997
d[8]: 0.9811
d[9]: 0.9973
d[10]: 0.99865
d[11]: 0.99973
d[12]: 0.999757


### Complexidade

Dado um grafo ponderado $G=(V,E)$, $|V|=n$ e $|E|=m$, a complexidade da nossa implementação se resume em:  

* *loop while* de $m$ arestas na *heap*: veja que caso um vértice já tenha sido visitado, não há o que fazer, pois o comando continue não avança nas operações do *loop*. Assim, teremos operações a fazer efetivamente $n$ vezes. No pior caso, gastaremos $n$ repetições multiplicado pelas operações de obtenção do topo da *heap* e atualização da estrutura (*constante* + $nlog\,n$) , busca dos vizinhos do vértice $u$ ($m$ operações, pelo Teorema do Aperto de Mãos) e atualização da *heap* ($log\,n$, no pior caso).

Logo, o Algoritmo de Dijkstra terá $O(mlog\,n + nlog\,n)$ operações.